# Word2Vec Model Training and Exploration

Ce notebook présente l'entraînement et l'exploration de modèles Word2Vec sur un corpus de textes des années 1950.

Nous allons:
1. Charger et préparer les données
2. Expérimenter avec différents paramètres (window size, min_count)
3. Évaluer les modèles avec similarity() et most_similar() 
4. Visualiser les embeddings

In [1]:
# Imports nécessaires
import os
import glob
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Configuration
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## 1. Chargement et préparation des données

Nous allons d'abord charger les fichiers texte du corpus des années 1950 et les préparer pour l'entraînement Word2Vec.

In [3]:
# Chemin vers les fichiers texte
data_dir = os.path.join(os.getcwd(), 'data', 'txt')

# Chargement des fichiers texte
def load_text_files(directory):
    texts = []
    for filepath in glob.glob(os.path.join(directory, '*.txt')):
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                texts.append(f.read())
        except UnicodeDecodeError:
            with open(filepath, 'r', encoding='latin-1') as f:
                texts.append(f.read())
    return texts

# Chargement des textes
texts = load_text_files(data_dir)
print(f"Nombre de fichiers texte chargés: {len(texts)}")

# Tokenisation en phrases
sentences = []
for text in texts:
    sentences.extend(sent_tokenize(text))
print(f"Nombre total de phrases: {len(sentences)}")

# Affichage de quelques phrases d'exemple
print("\nExemples de phrases:")
for i in range(min(3, len(sentences))):
    print(f"{i+1}. {sentences[i]}")

Nombre de fichiers texte chargés: 0
Nombre total de phrases: 0

Exemples de phrases:


## 2. Prétraitement et tokenisation

Nous allons maintenant tokeniser chaque phrase en mots et effectuer un prétraitement minimal.

In [ ]:
# Tokenisation des phrases en mots
def preprocess_sentence(sentence):
    # Tokenisation en mots
    tokens = word_tokenize(sentence.lower())
    # Filtrage basique (garder uniquement les mots)
    tokens = [token for token in tokens if token.isalnum()]
    return tokens

# Application du prétraitement à toutes les phrases
tokenized_sentences = [preprocess_sentence(sentence) for sentence in sentences]

# Filtrage des phrases vides
tokenized_sentences = [tokens for tokens in tokenized_sentences if tokens]

print(f"Nombre de phrases après prétraitement: {len(tokenized_sentences)}")
print("\nExemple de phrases tokenisées:")
for i in range(min(3, len(tokenized_sentences))):
    print(f"{i+1}. {tokenized_sentences[i]}")

## 3. Entraînement du modèle de référence

Nous allons d'abord entraîner un modèle Word2Vec avec des paramètres standard pour servir de référence.

In [ ]:
# Paramètres du modèle de référence
vector_size = 100
window = 5
min_count = 5

# Entraînement du modèle
model_ref = Word2Vec(sentences=tokenized_sentences,
                    vector_size=vector_size,
                    window=window,
                    min_count=min_count,
                    workers=4)

# Informations sur le modèle
print(f"Taille du vocabulaire: {len(model_ref.wv.key_to_index)}")
print(f"Dimension des vecteurs: {model_ref.wv.vector_size}")

# Exemple de mots les plus fréquents
print("\nMots les plus fréquents dans le vocabulaire:")
vocab_size = len(model_ref.wv.key_to_index)
sorted_words = sorted(model_ref.wv.key_to_index.items(), 
                     key=lambda x: model_ref.wv.get_vecattr(x[0], "count"),
                     reverse=True)
for word, _ in sorted_words[:10]:
    count = model_ref.wv.get_vecattr(word, "count")
    print(f"{word}: {count} occurrences")

## 4. Exploration des paramètres

Nous allons maintenant expérimenter avec différentes valeurs de `window` et `min_count` pour comprendre leur impact sur les résultats.

In [ ]:
# Définition des paramètres à tester
windows = [3, 5, 7]
min_counts = [3, 5, 10]

# Fonction pour entraîner un modèle avec des paramètres donnés
def train_model(sentences, window, min_count, vector_size=100):
    model = Word2Vec(sentences=sentences,
                    vector_size=vector_size,
                    window=window,
                    min_count=min_count,
                    workers=4)
    return model

# Entraînement des différents modèles
models = {}
for w in windows:
    for mc in min_counts:
        model_name = f"w{w}_mc{mc}"
        print(f"\nEntraînement du modèle {model_name}")
        model = train_model(tokenized_sentences, w, mc)
        models[model_name] = model
        print(f"Taille du vocabulaire: {len(model.wv.key_to_index)}")

# Comparaison des tailles de vocabulaire
print("\nComparaison des tailles de vocabulaire:")
vocab_sizes = {name: len(model.wv.key_to_index) for name, model in models.items()}
for name, size in vocab_sizes.items():
    print(f"{name}: {size} mots")

## 5. Évaluation des modèles

Nous allons maintenant évaluer les différents modèles en utilisant les fonctions `similarity()` et `most_similar()`.

In [ ]:
# Mots tests pour l'évaluation
test_words = ['france', 'paris', 'politique']
word_pairs = [('france', 'paris'), ('homme', 'femme'), ('roi', 'reine')]

# Fonction pour évaluer un modèle
def evaluate_model(model, name):
    print(f"\nÉvaluation du modèle {name}")
    
    # Test de most_similar
    print("\nMots les plus similaires:")
    for word in test_words:
        if word in model.wv:
            print(f"\nPour '{word}':")
            similars = model.wv.most_similar(word)
            for similar, score in similars[:5]:
                print(f"  {similar}: {score:.4f}")
                
    # Test de similarity
    print("\nSimilarités entre paires de mots:")
    for word1, word2 in word_pairs:
        if word1 in model.wv and word2 in model.wv:
            sim = model.wv.similarity(word1, word2)
            print(f"{word1} - {word2}: {sim:.4f}")

# Évaluation de tous les modèles
for name, model in models.items():
    evaluate_model(model, name)

## 6. Visualisation des embeddings

Pour mieux comprendre la structure de l'espace vectoriel, nous allons visualiser les embeddings en 2D à l'aide de t-SNE.

In [ ]:
# Sélection du meilleur modèle pour la visualisation
best_model = models['w5_mc5']  # On peut ajuster selon les résultats

def visualize_embeddings(model, n_words=100):
    # Sélection des mots les plus fréquents
    words = []
    vectors = []
    for word in list(model.wv.key_to_index.keys())[:n_words]:
        words.append(word)
        vectors.append(model.wv[word])
    
    # Réduction de dimension avec t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    vectors_2d = tsne.fit_transform(vectors)
    
    # Création du plot
    plt.figure(figsize=(15, 10))
    plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], alpha=0.3)
    
    # Ajout des labels
    for i, word in enumerate(words):
        plt.annotate(word, xy=(vectors_2d[i, 0], vectors_2d[i, 1]))
    
    plt.title("Visualisation t-SNE des embeddings Word2Vec")
    plt.grid(True)
    plt.show()

# Visualisation
visualize_embeddings(best_model)

## 7. Sauvegarde du meilleur modèle

Pour finir, nous allons sauvegarder le meilleur modèle pour une utilisation ultérieure.

In [ ]:
# Création du dossier models s'il n'existe pas
import os
models_dir = 'models'
os.makedirs(models_dir, exist_ok=True)

# Sauvegarde du meilleur modèle
model_path = os.path.join(models_dir, 'word2vec_best.model')
best_model.save(model_path)
print(f"Modèle sauvegardé dans {model_path}")

# Test de chargement
loaded_model = Word2Vec.load(model_path)
print("\nTest du modèle chargé:")
print("Most similar à 'france':", loaded_model.wv.most_similar('france')[:3])